<div class="alert alert-block alert-danger">

 <span style="color:red ; font-size:1.2em" > NOTE TO CODE REVIEWER: At 11am on Feb 22, my access to the web site I used to scrape my data was blocked. So from here forward you may not be able to run the code. The code was working fine last week. Most of the cells have the output from last week still here. <br> <br>
Try running the first code cell to see if you are given access to the website. If you are not, do not run the remaining cells.</span>
</div>

In [1]:
%reset -fs
import bs4
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np

<div class="alert alert-block alert-info">
<span style="font-size:1.2em" >In this notebook I am scraping the main table from  'the_numbers.com' web site that lists all films released in 2019 along with associated  data like revenues, genre etc.
 <br>
 <br>
 I also create url addresses to access the summary and casting page for each movie.  These pages provide more detailed information for each film.
 </span>
</div>

In [4]:
# Request html from web page
import requests
r = requests.get('https://www.the-numbers.com/market/2019/top-grossing-movies')
r

<Response [403]>

In [3]:
# take a look at the page

from IPython.core.display import display, HTML
display(HTML(r.text)) 

<div class="alert alert-block alert-danger">

 <span style="color:red ; font-size:1.2em" > NOTE TO CODE REVIEWER: Do not run code below if you were denied acces to website.</span>
</div>

In [8]:
# By examing the web site, I determined that the first table is the main table
# I find the table and convert to text. 
# I find all table headers to extract column names

s = bs(r.text)
table = s.find('table')
col = table.tr.find_all('th')
table.text


'\nRankMovieReleaseDateDistributorGenre2019 GrossTickets Sold\n\n1\nAvengers: Endgame\nApr 26, 2019\nWalt Disney\nAction\n$858,373,000\n93,708,843\n\n\n2\nThe Lion King\nJul 19, 2019\nWalt Disney\nAdventure\n$543,638,043\n59,349,131\n\n\n3\nFrozen II\nNov 22, 2019\nWalt Disney\nAdventure\n$450,439,533\n49,174,621\n\n\n4\nToy Story 4\nJun 21, 2019\nWalt Disney\nAdventure\n$434,038,008\n47,384,062\n\n\n5\nCaptain Marvel\nMar 8, 2019\nWalt Disney\nAction\n$426,829,839\n46,597,144\n\n\n6\nStar Wars: The Rise of Skyw…\nDec 20, 2019\nWalt Disney\nAdventure\n$390,706,234\n42,653,519\n\n\n7\nSpider-Man: Far From Home\nJul 2, 2019\nSony Pictures\nAction\n$390,532,085\n42,634,507\n\n\n8\nAladdin\nMay 24, 2019\nWalt Disney\nAdventure\n$355,559,216\n38,816,508\n\n\n9\nJoker\nOct 4, 2019\nWarner Bros.\nThriller/Suspense\n$333,772,511\n36,438,047\n\n\n10\nIt: Chapter Two\nSep 6, 2019\nWarner Bros.\nHorror\n$211,593,228\n23,099,697\n\n\n11\nJumanji: The Next Level\nDec 13, 2019\nSony Pictures\nAdvent

In [9]:
# Extract column names and put into a list
# Modify the names of the last two columns

col_list = []
for i in col:
    col_name = i.text
    col_list.append(col_name)

col_list[-1] = 'Tickets_Sold'
col_list[-2] = 'Revenues'
col_list

['Rank',
 'Movie',
 'ReleaseDate',
 'Distributor',
 'Genre',
 'Revenues',
 'Tickets_Sold']

In [10]:

num_cols = len(col_list)

In [11]:
"""
By examining the web site I note that the main table has 7 columns, the first
element of each row is film ranking. The first table element of the row after the 
last actual film listings is not numeric. So I construct a loop that selects 
the first table element and proceeds in steps of 7 until it no longer sees an element
that is numeric. The output from this loop is the number of rows in the table with
actual movie listings

I also note that the position 'index' for table elements is cumulative in that the
index doesn't reset with each row
"""

# Just checking my math 'indexing' to make sure I have the table structure right by extracting
# the last element of the last movie listing
table_elems = table.find_all('td')
table_elems[4759]

<td class="data">41</td>

In [12]:
#Loop through table to determine number of rows

num_rows = 0
for i in range(0,10000, 7):
    if (not table_elems[i].text.isnumeric()):
        num_rows = int(table_elems[i - 7].text)
        print(table_elems[i - 7].text)
        break
        
print(num_rows)
print(type(num_rows))

680
680
<class 'int'>


In [13]:
# Just a check to extract the first seven elements of the table which are the data for 
# the first movie in the first row

for i in range(0,7):
    print(table_elems[i].text)

1
Avengers: Endgame
Apr 26, 2019
Walt Disney
Action
$858,373,000
93,708,843


In [14]:
# Another check to see if I can extract the url associated with the name of each movie
# that navigates to the summary page

table_elems[1].find(href = True)
u = table_elems[1].find(href = True)
u['href']
# type(table_elems[1].find('href'))


'/movie/Avengers-Endgame-(2019)#tab=summary'

In [15]:
# Use the col_list I created with elements corresponding to data column names to
# create a dictionary with column names as keys

my_dict = {key:[] for key in col_list}
my_dict

{'Rank': [],
 'Movie': [],
 'ReleaseDate': [],
 'Distributor': [],
 'Genre': [],
 'Revenues': [],
 'Tickets_Sold': []}

In [5]:
# Loop over the dictionary I created selecting each key (column name)
# Use an inner while loop to iterate over all rows of the table in steps of 7 to
# extract data for eack key(column name) for all rows


for i, name in enumerate(my_dict):
    j = i
    while (j < (num_rows * num_cols)):
        my_dict[name].append(table_elems[j].text)
        j += 7

NameError: name 'my_dict' is not defined

In [17]:
# Create a dataframe from the dictionary above
df = pd.DataFrame(my_dict)

In [18]:
df.head(5)

,Rank,Movie,ReleaseDate,Distributor,Genre,Revenues,Tickets_Sold
0,1,Avengers: Endgame,"Apr 26, 2019",Walt Disney,Action,"$858,373,000","93,708,843"
1,2,The Lion King,"Jul 19, 2019",Walt Disney,Adventure,"$543,638,043","59,349,131"
2,3,Frozen II,"Nov 22, 2019",Walt Disney,Adventure,"$450,439,533","49,174,621"
3,4,Toy Story 4,"Jun 21, 2019",Walt Disney,Adventure,"$434,038,008","47,384,062"
4,5,Captain Marvel,"Mar 8, 2019",Walt Disney,Action,"$426,829,839","46,597,144"


In [23]:
# Create a list of urls that navigate to the summary info page for each movie by
# noting the structure of the website addressing scheme and appending the film name
# extracted from the main table

url_summary_list = []
url_pre = 'https://www.the-numbers.com'

i = 1
while(i < (num_rows * num_cols)):
    url_href = table_elems[i].find(href = True)['href']
    full_url = url_pre + url_href
    url_summary_list.append(full_url)
    i += 7

#Examine the list
url_summary_list

['https://www.the-numbers.com/movie/Avengers-Endgame-(2019)#tab=summary',
 'https://www.the-numbers.com/movie/Lion-King-The-(Live-Action)-(2019)#tab=summary',
 'https://www.the-numbers.com/movie/Frozen-II-(2019)#tab=summary',
 'https://www.the-numbers.com/movie/Toy-Story-4-(2019)#tab=summary',
 'https://www.the-numbers.com/movie/Captain-Marvel-(2019)#tab=summary',
 'https://www.the-numbers.com/movie/Star-Wars-The-Rise-of-Skywalker-(2019)#tab=summary',
 'https://www.the-numbers.com/movie/Spider-Man-Far-From-Home-(2019)#tab=summary',
 'https://www.the-numbers.com/movie/Aladdin-(2019)#tab=summary',
 'https://www.the-numbers.com/movie/Joker-(2019)#tab=summary',
 'https://www.the-numbers.com/movie/It-Chapter-Two-(2019)#tab=summary',
 'https://www.the-numbers.com/movie/Jumanji-The-Next-Level#tab=summary',
 'https://www.the-numbers.com/movie/Us-(2019)#tab=summary',
 'https://www.the-numbers.com/movie/Fast-and-Furious-Presents-Hobbs-and-Shaw-(2019)#tab=summary',
 'https://www.the-numbers.com/m

In [37]:
# Just a check of radom rows to make sure the extractiion and url address work
url_summary_list[150]

'https://www.the-numbers.com/movie/Liu-Lang-Di-Qiu-(China)-(2019)#tab=summary'

In [32]:
#Repeat the same process as above to create url addresses for 
#accessing the casting page

url_cast_list = []
for i in range(num_rows):
    cast_url = url_summary_list[i]
    cast_url = cast_url.replace('#tab=summary','#tab=cast-and-crew')
    url_cast_list.append(cast_url)

url_cast_list

680

In [45]:
#Add the summary and casting page urls to dataframe as new columns

df['summary_page_url'] = url_summary_list
url = df.loc[150, 'summary_page_url'] #random checks to make sure the new columns worked
url

'https://www.the-numbers.com/movie/Liu-Lang-Di-Qiu-(China)-(2019)#tab=summary'

In [35]:
df['cast_page_url'] = url_cast_list
df

,Rank,Movie,ReleaseDate,Distributor,Genre,Revenues,Tickets_Sold,summary_page_url,cast_page_url
0,1,Avengers: Endgame,"Apr 26, 2019",Walt Disney,Action,"$858,373,000","93,708,843",https://www.the-numbers.com/movie/Avengers-End...,https://www.the-numbers.com/movie/Avengers-End...
1,2,The Lion King,"Jul 19, 2019",Walt Disney,Adventure,"$543,638,043","59,349,131",https://www.the-numbers.com/movie/Lion-King-Th...,https://www.the-numbers.com/movie/Lion-King-Th...
2,3,Frozen II,"Nov 22, 2019",Walt Disney,Adventure,"$450,439,533","49,174,621",https://www.the-numbers.com/movie/Frozen-II-(2...,https://www.the-numbers.com/movie/Frozen-II-(2...
3,4,Toy Story 4,"Jun 21, 2019",Walt Disney,Adventure,"$434,038,008","47,384,062",https://www.the-numbers.com/movie/Toy-Story-4-...,https://www.the-numbers.com/movie/Toy-Story-4-...
4,5,Captain Marvel,"Mar 8, 2019",Walt Disney,Action,"$426,829,839","46,597,144",https://www.the-numbers.com/movie/Captain-Marv...,https://www.the-numbers.com/movie/Captain-Marv...
...,...,...,...,...,...,...,...,...,...
675,676,Chef Flynn,"Nov 9, 2018",Kino Lorber,Documentary,$721,78,https://www.the-numbers.com/movie/Chef-Flynn-(...,https://www.the-numbers.com/movie/Chef-Flynn-(...
676,677,I am Not a Witch,"Sep 7, 2018",Film Movement,Drama,$550,60,https://www.the-numbers.com/movie/I-am-Not-a-W...,https://www.the-numbers.com/movie/I-am-Not-a-W...
677,678,Divide and Conquer: The Sto…,"Dec 7, 2018",Magnolia Pictures,Documentary,$474,51,https://www.the-numbers.com/movie/Divide-and-C...,https://www.the-numbers.com/movie/Divide-and-C...
678,679,Senso,"Jul 8, 1968",Fleetwood Films,Drama,$437,47,https://www.the-numbers.com/movie/Senso-(Italy...,https://www.the-numbers.com/movie/Senso-(Italy...


<div class="alert alert-block alert-danger">

 <span style="color:red ; font-size:1.2em" > NOTE TO CODE REVIEWER: Do not run the cells below unless you were given access to the website and ran the cells above.</span>
</div>

In [36]:
#Store dataframe to a pickle file in the same directory as this notebool

# import pickle
# pd.to_pickle(df, 'df_all_movies_2019_pk')